In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import re
import pickle
import torch
from tqdm import tqdm
import math
from collections import Counter
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch.nn as nn
import transformers
import tweetnlp
from inference.engine import Model

2023-08-19 22:40:12.764590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 22:40:13.472975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/csgrad/sougatas/environments/dev/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-08-19 22:40:19,092	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-19 22:40:19,145	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 fo

In [ ]:
import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [ ]:
[i for i in os.listdir(task4_path) if i.endswith(".csv") and i.startswith("train")]

['train_A_AH_HASOC2023.csv',
 'train_BE_AH_HASOC2023.csv',
 'train_BO_AH_HASOC2023.csv']

In [ ]:
lang_map = {'A': {'src':'asm_Beng', 'dest': ['ben_Beng', 'brx_Deva']},
            'BE': {'src':'ben_Beng', 'dest': ['asm_Beng', 'brx_Deva']},
            'BO': {'src':'brx_Deva', 'dest': ['asm_Beng', 'ben_Beng']}
           }

## Translations

We translate the data for each language to the other languages and add to their training set

In [ ]:
from inference.engine import Model

ind_en_model = Model("~/IndicTrans2/indic-en-preprint/fairseq_model/",
                     model_type="fairseq")
en_ind_model = Model("~/IndicTrans2/en-indic-preprint/fairseq_model/",
                     model_type="fairseq")

Initializing sentencepiece model for SRC and TGT
Initializing model for translation


2023-08-19 14:56:12 | INFO | fairseq.tasks.translation | [SRC] dictionary: 122706 types
2023-08-19 14:56:12 | INFO | fairseq.tasks.translation | [TGT] dictionary: 32296 types


Initializing sentencepiece model for SRC and TGT
Initializing model for translation


2023-08-19 14:56:31 | INFO | fairseq.tasks.translation | [SRC] dictionary: 32322 types
2023-08-19 14:56:31 | INFO | fairseq.tasks.translation | [TGT] dictionary: 122672 types


In [ ]:
import demoji
# demoji.findall(tweet)

In [ ]:
task4_path = "~/hasoc_2023_ub/datasets/task4_annihilate/"

data_df, trans_df = pd.DataFrame(), pd.DataFrame()
""" Indic to English """
for file in tqdm([i for i in os.listdir(task4_path) if i.endswith(".csv") and i.startswith("train")]):
    print("PROCESSING FILE:",file)
    tmpdf = pd.read_csv(task4_path+file)
    split, lang, _, _ = file.split("/")[-1].split("_")
    tmpdf["split"], tmpdf["lang"] = split, lang
    tmpdf["truncated_text"] = tmpdf["text"].apply(lambda x : " ".join(x.split(" ")[:50]))
    data_df = pd.concat([data_df, tmpdf]).reset_index(drop=True)

    """ TRANSLATIONS """
    sents, labels = list(tmpdf["truncated_text"]), list(tmpdf["task_1"])

    eng_trans = [ind_en_model.translate_paragraph(i, lang_map[lang]["src"], "eng_Latn") for i in tqdm(sents)]
    tdf = pd.DataFrame(sents, columns=["src_text"])
    tdf["translated_text"], tdf["src_lang"], tdf["dest_lang"], tdf["label"] = eng_trans, lang_map[lang]["src"], \
                                                                                            "eng_Latn", labels

    assert len(sents) == len(labels) == len(eng_trans)
    trans_df = pd.concat([trans_df, tdf]).reset_index(drop=True)

    """ English to Indic """
    for dest in lang_map[lang]["dest"]:
        trans = [en_ind_model.translate_paragraph(i, "eng_Latn", dest) for i in tqdm(eng_trans)]
        assert len(trans) == len(labels) == len(eng_trans)
        tdf = pd.DataFrame(eng_trans, columns=["src_text"])
        tdf["translated_text"], tdf["src_lang"], tdf["dest_lang"], tdf["label"] = trans, "eng_Latn", dest, labels
        trans_df = pd.concat([trans_df, tdf]).reset_index(drop=True)
    print("DONE PROCESSING FILE:",file,"\n","====="*5,"\n")
data_df.shape, trans_df.shape

In [ ]:
data_df.head(1)

,S. No.,text,task_1,split,lang,truncated_text
0,1,লাগিলে ইযাৰ পৰাই কলি লাগাম,NOT,train,A,লাগিলে ইযাৰ পৰাই কলি লাগাম


In [ ]:
trans_df.head(1)

,src_text,translated_text,src_lang,dest_lang,label
0,লাগিলে ইযাৰ পৰাই কলি লাগাম,If only I could get a call from him,asm_Beng,eng_Latn,NOT


In [ ]:
trans_df.groupby(["src_lang", "dest_lang"]).agg({"src_text": "count"})

src_text
src_lang dest_lang          
asm_Beng eng_Latn       4036
ben_Beng eng_Latn       1281
brx_Deva eng_Latn       1679
eng_Latn asm_Beng       2960
         ben_Beng       5715
         brx_Deva       5317

In [ ]:
data_df.shape, trans_df.shape

((6996, 6), (20988, 5))

In [ ]:
data_df.task_1.value_counts()

HOF    3860
NOT    3136
Name: task_1, dtype: int64

In [ ]:
dct = {"data_df": data_df, "trans_df": trans_df}
pickle.dump(dct, open("./all_traslations.pkl", "wb"))

## Modelling

In [ ]:
# !pip install tweetnlp

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [ ]:
trans_df["translated"] = True
data_df["translated"] = False

In [ ]:
trans_df.head(3)

,src_text,translated_text,src_lang,dest_lang,label,translated
0,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দি...,But I have heard that Mamata Banerjee has remo...,ben_Beng,eng_Latn,NOT,True
1,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদ...","Every minister of every party, MLA, MP, they h...",ben_Beng,eng_Latn,NOT,True
2,বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে ব...,"Universities, seminaries, publishing houses, a...",ben_Beng,eng_Latn,HOF,True


In [ ]:
data_df.head(1)

,S. No.,text,task_1,split,lang,cleansed_truncated_text,translated
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দি...,NOT,train,BE,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দি...,False


In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn import metrics

# Training function: Performs forward propagation, backpropagation & optimization.
# We also implement gradient clipping, which prevents the gradients from exploding

def train(model, dataloader, optimizer, criterion, clip=1.0, v1=True):

    model.train()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for batch in tqdm(dataloader):
        batch = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()
        if v1:
            x_in_sam, x_attn_sam, y = batch
            output = model(x_in_sam, x_attn_sam)
        else:
            x_in_sam, x_attn_sam, x_in_deva, x_attn_deva, y = batch
            output = model(x_in_sam, x_attn_sam, x_in_deva, x_attn_deva)

        loss = criterion(output.view(-1), y)
        loss.backward()

        #gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        ep_t_loss += loss.item()
        batch_num += 1
        pred.extend((torch.sigmoid(output.view(-1)) >= 0.5).tolist())
        tgt.extend(y.tolist())

    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro')

def evaluate(model, dataloader, criterion, v1=True):

    model.eval()

    ep_t_loss = 0
    batch_num  = 0
    pred, tgt = [], []
    for batch in tqdm(dataloader):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            if v1:
                x_in_sam, x_attn_sam, y = batch
                output = model(x_in_sam, x_attn_sam)
            else:
                x_in_sam, x_attn_sam, x_in_deva, x_attn_deva, y = batch
                output = model(x_in_sam, x_attn_sam, x_in_deva, x_attn_deva)

            loss = criterion(output.view(-1), y)

            ep_t_loss += loss.item()
            batch_num += 1
            pred.extend((torch.sigmoid(output.view(-1)) >= 0.5).tolist())
            tgt.extend(y.tolist())

    return ep_t_loss/batch_num, metrics.f1_score(tgt, pred, average='macro'), pred, tgt

class Classifier(nn.Module):
    def __init__(self, base1, base2=None):
        super().__init__()
        self.base1 = base1
        self.o1_drop = nn.Dropout(0.1)
        if base2 is not None:
            self.base2 = base2
            self.o2_drop = nn.Dropout(0.1)
        else:
            self.base2 = None

        if base2 is not None:
            self.linear = nn.Linear(base1.config.hidden_size * 2, 1)
        else:
            self.linear = nn.Linear(base1.config.hidden_size, 1)

    def forward(self, in_sam, attn_sam, in_deva=None, attn_deva=None):
        o1 = self.o1_drop(self.base1(input_ids=in_sam, attention_mask=attn_sam)["pooler_output"])
        if self.base2 is not None and in_deva is not None:
            o2 = self.o2_drop(self.base2(input_ids=in_deva, attention_mask=attn_deva)["pooler_output"])
            o = torch.cat([o1, o2], -1)
            return self.linear(o)
        else:
            return self.linear(o1)

def get_lang_dataset(lang_abbr):
    t1 = trans_df[trans_df["dest_lang"] == lang_map[lang_abbr]["src"]][["translated_text", "translated", "label"]]
    t2 = data_df[data_df["lang"] == lang_abbr][["cleansed_truncated_text", "translated", "task_1"]]
    t1.columns = ["text", "translated", "label"]
    t2.columns = ["text", "translated", "label"]
    t = pd.concat([t1, t2])
    t["lang"] = lang_abbr
    t = t.drop_duplicates().reset_index(drop=True)
    return t

def get_desc(row):
    if row["lang"] == "A":
        if row["translated"]:
            return "assamese translated:"
        else:
            return "assamese original:"
    if row["lang"] == "BE":
        if row["translated"]:
            return "bengali translated:"
        else:
            return "bengali original:"
    if row["lang"] == "BO":
        if row["translated"]:
            return "bodo translated:"
        else:
            return "bodo original:"

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
data_df.columns

Index(['S. No.', 'text', 'task_1', 'split', 'lang', 'cleansed_truncated_text',
       'translated'],
      dtype='object')

In [ ]:
asm_df = get_lang_dataset("A")
ben_df = get_lang_dataset("BE")
bdo_df = get_lang_dataset("BO")
train_df = ben_df#pd.concat([asm_df, ben_df, bdo_df]).drop_duplicates().reset_index(drop=True)

print(asm_df.shape, ben_df.shape, bdo_df.shape, train_df.shape)

train_df["label"] = train_df.label.apply(lambda x : 1 if x == "HOF" else 0)
train_df["description"] = train_df.apply(get_desc, 1)
train_df = train_df.sample(frac=1.0)
vl_ = 500
val_df = train_df[:vl_].sample(frac=1.0).reset_index(drop=True)
train_df = train_df[vl_:].sample(frac=1.0).reset_index(drop=True)
print(train_df.shape, val_df.shape)

(6970, 4) (6952, 4) (6953, 4) (6952, 4)
(6452, 5) (500, 5)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_model_dict = {}

In [ ]:
for lng in ['BE']:#['A', 'BE', 'BO']:
    tr_df = train_df[(train_df.lang == lng) & (~train_df.translated)].reset_index(drop=True)
    vl_df = val_df[(val_df.lang == lng) & (~val_df.translated)].reset_index(drop=True)

    tr_X, vl_X = list(tr_df.text), list(vl_df.text)
    tr_Y, vl_Y = list(tr_df.label), list(vl_df.label)

    tf_vec = CountVectorizer()
    X_tr_tf = tf_vec.fit_transform(tr_X)
    X_vl_tf = tf_vec.transform(vl_X)

    nb_ = MultinomialNB()
    nb_.fit(X_tr_tf, tr_Y)
    vl_pred = nb_.predict(X_vl_tf)
    print("FOR ",lng,":\n",metrics.classification_report(vl_Y, vl_pred),"==="*5,"\n")
    nb_model_dict[lng] = {"tf_vec": tf_vec, "nb_": nb_}

FOR  BE :
               precision    recall  f1-score   support

           0       0.68      0.80      0.73        65
           1       0.59      0.43      0.50        44

    accuracy                           0.65       109
   macro avg       0.63      0.62      0.62       109
weighted avg       0.64      0.65      0.64       109



In [ ]:
run = "baseline_beng"#"deva"#"baseline"#"sam_deva"#"baseline"

In [ ]:
tokenizer_sam = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-Sam-TLM")
model_sam = AutoModel.from_pretrained("ai4bharat/IndicBERTv2-MLM-Sam-TLM")
model_deva = None

if "deva" in run:
    tokenizer_deva = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-SS")
    model_deva = AutoModel.from_pretrained("ai4bharat/IndicBERTv2-SS")

if "deva" in run and "sam" in run:
    print("Multi input model")
    classifier = Classifier(model_sam, model_deva)
elif "sam" in run:
    print("only sam input model")
    classifier = Classifier(model_sam)
elif "deva" in run:
    print("only deva input model")
    classifier = Classifier(model_deva)
else:
    print("only sam input model")
    classifier = Classifier(model_sam)

print(f'The model has {count_parameters(classifier):,} trainable parameters')

Some weights of the model checkpoint at ai4bharat/IndicBERTv2-MLM-Sam-TLM were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


only sam input model
The model has 278,042,113 trainable parameters


In [ ]:
train_sam_toks = tokenizer_sam(list(train_df["description"] + train_df["text"]), max_length=50, truncation=True,
                               return_tensors="pt", padding=True)
val_sam_toks = tokenizer_sam(list(val_df["description"] + val_df["text"]), max_length=50, truncation=True,
                             return_tensors="pt", padding=True)
if "deva" in run:
    train_deva_toks = tokenizer_deva(list(train_df["description"] + train_df["text"]), max_length=50, truncation=True,
                                     return_tensors="pt", padding=True)
    val_deva_toks = tokenizer_deva(list(val_df["description"] + val_df["text"]), max_length=50, truncation=True,
                                   return_tensors="pt", padding=True)

In [ ]:
train_sam_toks["input_ids"].shape, val_sam_toks["input_ids"].shape

(torch.Size([6452, 50]), torch.Size([500, 50]))

In [ ]:
Y_train, Y_val = torch.tensor(train_df["label"])*1.0, torch.tensor(val_df["label"])*1.0
Y_train.shape, Y_val.shape

(torch.Size([6452]), torch.Size([500]))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Below we define a function to create train, test & valid dataloaders in Pytorch

batch_size = 16

def get_dataloader(input_ids, attention_mask, y, train=True):
    data = TensorDataset(input_ids, attention_mask, y)
    sampler = RandomSampler(data) if train else SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

def get_dataloader_v2(sam_input_ids, sam_attention_mask,
                      deva_input_ids, deva_attention_mask, y, train=True):
    data = TensorDataset(sam_input_ids, sam_attention_mask,
                         deva_input_ids, deva_attention_mask, y)
    sampler = RandomSampler(data) if train else SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

# train_dataloader = get_dataloader(train_sam_toks["input_ids"], train_sam_toks["attention_mask"], Y_train)
# val_dataloader = get_dataloader(val_sam_toks["input_ids"], val_sam_toks["attention_mask"], Y_val)

# train_dataloader = get_dataloader_v2(train_sam_toks["input_ids"], train_sam_toks["attention_mask"],
#                                      train_deva_toks["input_ids"], train_deva_toks["attention_mask"], Y_train)
# val_dataloader = get_dataloader_v2(val_sam_toks["input_ids"], val_sam_toks["attention_mask"],
#                                    val_deva_toks["input_ids"], val_deva_toks["attention_mask"], Y_val)

if "deva" in run and "sam" in run:
    print("Multi input dataloader")
    train_dataloader = get_dataloader_v2(train_sam_toks["input_ids"], train_sam_toks["attention_mask"],
                                     train_deva_toks["input_ids"], train_deva_toks["attention_mask"], Y_train)
    val_dataloader = get_dataloader_v2(val_sam_toks["input_ids"], val_sam_toks["attention_mask"],
                                       val_deva_toks["input_ids"], val_deva_toks["attention_mask"], Y_val)
elif "sam" in run:
    print("only sam input dataloader")
    train_dataloader = get_dataloader(train_sam_toks["input_ids"], train_sam_toks["attention_mask"], Y_train)
    val_dataloader = get_dataloader(val_sam_toks["input_ids"], val_sam_toks["attention_mask"], Y_val)
elif "deva" in run:
    print("only deva input dataloader")
    train_dataloader = get_dataloader(train_deva_toks["input_ids"], train_deva_toks["attention_mask"], Y_train)
    val_dataloader = get_dataloader(val_deva_toks["input_ids"], val_deva_toks["attention_mask"], Y_val)
else:
    print("only sam input dataloader")
    train_dataloader = get_dataloader(train_sam_toks["input_ids"], train_sam_toks["attention_mask"], Y_train)
    val_dataloader = get_dataloader(val_sam_toks["input_ids"], val_sam_toks["attention_mask"], Y_val)

only sam input dataloader


In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optim = torch.optim.AdamW(classifier.parameters(), lr = 2e-5)

In [ ]:
best_valid_loss = float('inf')
tot_t_loss, tot_v_loss =[],[]
N_EPOCHS = 20
device = "cuda:3"
early_stopping_marker = []
early_stopping = 2

In [ ]:
classifier = classifier.to(device)
mname = "model_least_loss_08192023_BE.pt" if run == "baseline" else run+"_model_least_loss_08192023_BE.pt"
v1 = False if "deva" in run and "sam" in run else True
print("mname:", mname, "| v1:", v1)

mname: baseline_beng_model_least_loss_08192023_BE.pt | v1: True


In [ ]:
import time
from tqdm import tqdm

for epoch in tqdm(range(N_EPOCHS)):

    tr_l, tr_f1= train(classifier, train_dataloader, optim, criterion, v1=v1)
    tot_t_loss.append(tr_l)

    val_l, val_f1, pred, tgt = evaluate(classifier, val_dataloader, criterion, v1=v1)
    tot_v_loss.append(val_l)

    if val_l <= best_valid_loss:
        best_valid_loss = val_l
        best_pred, best_tgt = pred, tgt
        torch.save(classifier.state_dict(), mname)
        print("\nBest Model Saved !!")
        early_stopping_marker.append(False)
    else:
        early_stopping_marker.append(True)

    print("\n")
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Total Loss: {tr_l:.3f} | Train F1: {tr_f1:.3f}')
    print(f'\tVal. Total Loss: {val_l:.3f} | Valid F1: {val_f1:.3f}')
    print("_________________________________________________________________")
    if all(early_stopping_marker[-early_stopping:]) and len(early_stopping_marker) >= early_stopping:
        print("Early stopping training as the Val loss did NOT improve for the last "+str(early_stopping)+" epochs.")
        break

In [ ]:
classifier.load_state_dict(torch.load(mname))
classifier.eval()
print(mname,"Model Loaded!")

baseline_beng_model_least_loss_08192023_BE.pt Model Loaded!


In [ ]:
# run = "sam_deva"#"baseline"
run

'baseline'

In [ ]:
nb_pred_

array([0.17308353, 0.85997318, 0.07644555, ..., 0.76766401, 0.6153633 ,
       0.58595888])

In [ ]:
pth = "~/hasoc_2023_ub/datasets/task4_annihilate/"
version = "4-deva-" #""
for i in os.listdir(pth):
    if "test" in i and "chetona_pred" not in i and i.split("_")[1] == "BE":
        print("Running predictions for",i)
        tdf_ = pd.read_csv(pth+i)
        tdf_["lang"], tdf_["translated"] = i.split("_")[1], False
        tdf_["description"] = tdf_.apply(get_desc, 1)

#         sam_toks = tokenizer_sam(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
#                              return_tensors="pt", padding=True)
        if "deva" in run and "sam" in run:
            sam_toks = tokenizer_sam(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
                             return_tensors="pt", padding=True)
            deva_toks = tokenizer_deva(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
                                     return_tensors="pt", padding=True)
            data_ = TensorDataset(sam_toks["input_ids"], sam_toks["attention_mask"],
                                 deva_toks["input_ids"], deva_toks["attention_mask"])
        elif "sam" in run:
            sam_toks = tokenizer_sam(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
                             return_tensors="pt", padding=True)
            deva_toks = None
            data_ = TensorDataset(sam_toks["input_ids"], sam_toks["attention_mask"])
        elif "deva" in run:
            deva_toks = tokenizer_deva(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
                             return_tensors="pt", padding=True)
            sam_toks = None
            data_ = TensorDataset(deva_toks["input_ids"], deva_toks["attention_mask"])
        else:
            sam_toks = tokenizer_sam(list(tdf_["description"] + tdf_["text"]), max_length=50, truncation=True,
                             return_tensors="pt", padding=True)
            deva_toks = None
            data_ = TensorDataset(sam_toks["input_ids"], sam_toks["attention_mask"])

        dataloader_ = DataLoader(data_, sampler=SequentialSampler(data_), batch_size=batch_size)

        """ Make predictions """
        nb_pred_ = nb_model_dict[tdf_.lang.iloc[0]]["nb_"].predict_proba(\
                            nb_model_dict[tdf_.lang.iloc[0]]["tf_vec"].transform(list(tdf_.text)))[:,-1]
        pred_ = []
        for batch_ in tqdm(dataloader_):
            batch_ = tuple(t.to(device) for t in batch_)
            if "deva" in run and "sam" in run:
                sam_in_, sam_attn_, deva_in_, deva_attn_ = batch_
                with torch.no_grad():
                    output_ = classifier(in_sam=sam_in_, attn_sam=sam_attn_,
                                         in_deva=deva_in_, attn_deva=deva_attn_)
            else:
                sam_in_, sam_attn_ = batch_
                deva_in_, deva_attn_ = None, None
                with torch.no_grad():
                    output_ = classifier(in_sam=sam_in_, attn_sam=sam_attn_,
                                         in_deva=deva_in_, attn_deva=deva_attn_)
            pred_.extend((torch.sigmoid(output_.view(-1))).tolist())

        pred_ = (np.asarray(pred_)>= 0.5).tolist()#(((0.8*np.asarray(pred_) + 0.2*nb_pred_)) >= 0.5).tolist()
        fop_ = pth+i.replace(".csv", "_"+run+"_ver_"+version+"_chetona_pred.csv")
        tdf_["pred"] = ["HOF" if i else "NOT" for i in pred_]
        tdf_["pred_nb"] = ["HOF" if i else "NOT" for i in nb_pred_]
        tdf__ = tdf_[['S. No.', 'pred']]
        tdf__.columns = ["S. No.", "task_1"]
        print(tdf__.shape, "Predictions done for:", i)
        print("Output location:", fop_)

        tdf__.to_csv(fop_, index=False)
        # break